In [1]:
# Getting the right libraries -- ignore this part
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm

C:\Users\Eier\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
# reading the data
from numpy import genfromtxt
my_data = genfromtxt('https://stanford.edu/~lodewijk/share/heart.csv', delimiter=',', skip_header=1)

In [3]:
# Extracting X and y
X = np.array([x[:-1] for x in my_data])
y = np.array([x[-1] for x in my_data])
print (X.shape)
print (y.shape)

(303, 13)
(303,)


In [4]:
# Partitioning into training and test
np.random.seed(42)
rnds = np.random.rand(303) > 0.2
X_train = X[rnds]
X_test = X[~rnds]
y_train = y[rnds]
y_test = y[~rnds]
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(236, 13)
(67, 13)
(236,)
(67,)


In [5]:
# HW Assignment (extra credit) 

# (a) Run SVM on X_train and y_train and report the accuracy on X_train and X_test (1.5% credit). Pick a value of C that give good results.
# (b) Run QP as opposed to SVM and verify predictions are the same (1% credit). See https://www.cvxpy.org/ or  https://cvxopt.org/examples/tutorial/qp.html .
#     Note that the parameter C may not cleanly correspond to 1/(2*lambda) because of randomness, so it is ok if the exact QP is not the same, but you should
#     be able to get the predictions to match.
#
# FAQ 1: I don't know python. What should I do? Answer: you have most of what you need here or at cvxpy
#
# FAQ 2: Can I use another language? Answer: Yes, if you can find a QP solver. Just do part (b) and you will get the full 2.5%. 
#
# FAQ 3: How much support can I expect?
#          Answer: On the theory part, as much as you need
#                  On the assignment: we will demo how to solve a QP using cvxpy at some point, but that's all the help you can expect.
#
# FAQ 4: How do I install numpy? Where do I get python? How do I install cvxpy or cvxopt?
# Answer: Don't, don't and don't. Go to the colaboratory (https://colab.research.google.com) and start a new Python2 notebook, and then just import the libraries, e.g. on the first line in this notebook

# FAQ 5: What should I submit? Answer: Please upload a pdf printout of your notebook. Include a link to your workbook (make it publicly viewable at least by Stanford accounts) in the pdf
#
# students who are struggling can catch up and students who want more challenging work can focus on the two modules.
# Final Note: You can collaborate on getting cvxpy to work and on getting colaborotary and svm libraries to work, and on reading data, but not on the actual training and prediction tasks.

## (a)

From the ppt: "Since this for this data set we our algorithm to be conservative (i.e., always correctly classify individuals with heart disease) we may want to modify our objective accordingly." 

Since we want to be conservative I have decided to use the recall/sensitivity as my objective. This is the fraction of positives that are correctly identified. 

In [6]:
from sklearn.metrics import accuracy_score, recall_score

In [7]:
param_grid = np.arange(0.1, 10, 0.1) #Range of possible C values from 0.1 to 9.9.

recall_scores = {} #Store the recall scores
for i in param_grid: #Loop over all the possible values for C
    svc = svm.SVC(C = i, kernel = "linear")
    svc.fit(X_train,y_train)
    y_pred = svc.predict(X_test)
    recall_scores[i] = recall_score(y_pred,y_test)


In [8]:
c = max(recall_scores, key = recall_scores.get)
print(c)
svc = svm.SVC(C = c, kernel = "linear")
svc.fit(X_train,y_train)

6.7


SVC(C=6.7, kernel='linear')

In [9]:
y_pred = svc.predict(X_test)

print(accuracy_score(y_pred,y_test))
print(recall_score(y_pred,y_test))

0.8955223880597015
0.8809523809523809


## (b)

In [10]:
from cvxopt import matrix, solvers

In [11]:
d = len(X_train[0]) #Total number of features
n = len(X_train) #Total number of data points/deltas

m = sum(y_train)
k = len(y_train)-m

n_vars = 1 + d + n

In [12]:
lmb = 1/(2*c)
print(lmb)

Q = []
p = []

for i in range(n_vars):
    q_row = [0.0] * n_vars 
    if i <= d: 
        p.append(0.0)
        if i <= 1:
            q_row[i] = 2.0*lmb
    else:
        p.append(1.0)

    Q.append(q_row)
    
G = []
h = []
for i in range(n):
    if i < m:
        h.append(-1.0)
        h.append(0)
        g1 = [1] + list(X_train[i]) + [0.0] * n
        g1[1+d+i] = -1.0
        g2 = [0.0] * n_vars
        g2[1+d+i] = -1.0
    else:
        h.append(-1.0)
        h.append(0)
        g1 = [-1] + list(-X_train[i]) + [0.0] * n
        g1[1+d+i] = -1.0
        g2 = [0.0] * n_vars
        g2[1+d+i] = -1.0
    
    G.append(g1)
    G.append(g2)
    
solvers.options['show_progress']=True
solution = solvers.qp(matrix(Q), matrix(p), matrix(G).trans(), matrix(h), verbose=True)

betas_qp = np.array(solution['x'][1:14]).reshape(13)
alpha_qp = solution['x'][0]

0.07462686567164178
     pcost       dcost       gap    pres   dres
 0: -1.2116e+02  5.4845e+02  2e+03  4e+00  7e+02
 1:  2.7560e+02 -6.7976e+01  5e+02  5e-01  8e+01
 2:  1.2672e+02  6.2471e+01  7e+01  6e-02  9e+00
 3:  9.9202e+01  8.0291e+01  2e+01  1e-02  2e+00
 4:  9.2562e+01  8.5883e+01  7e+00  4e-03  7e-01
 5:  9.0739e+01  8.7598e+01  3e+00  1e-03  2e-01
 6:  8.9823e+01  8.8203e+01  2e+00  3e-04  6e-02
 7:  8.9127e+01  8.8756e+01  4e-01  6e-05  1e-02
 8:  8.8985e+01  8.8870e+01  1e-01  1e-05  2e-03
 9:  8.8931e+01  8.8915e+01  2e-02  1e-06  2e-04
10:  8.8923e+01  8.8922e+01  1e-03  8e-08  1e-05
11:  8.8922e+01  8.8922e+01  1e-05  8e-10  1e-07
12:  8.8922e+01  8.8922e+01  1e-07  8e-12  1e-09
Optimal solution found.


In [13]:
def make_prediction(X, alpha, betas):
    y_pred = []
    for i in range(len(X)):
        val = alpha + np.dot(X[i], betas)
        if val < 0:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred


y_pred_qp = make_prediction(X_test, alpha_qp, betas_qp)

print(accuracy_score(y_pred_qp, y_test))
print(recall_score(y_pred_qp, y_test))

0.8656716417910447
0.8571428571428571


In [14]:
sum(abs(y_pred_qp-y_pred))

2.0

In [15]:
#As we can see in the cell above, we have only two predictions that differ from the sklearn library and the quadratic programming version